In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras import layers, ops
#from tensorflow_docs.vis import embed
#import tensorflow_addons as tfa
from itertools import zip_longest
import shutil
import os
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.utils import make_grid
import PIL

from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
autotune = tf.data.AUTOTUNE
import os
#os.environ["KERAS_BACKEND"]= "tensorflow"
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [18]:
#Load datasets into monet and photo, gotten from https://www.kaggle.com/code/sai10py/gans-art-creation
input_img_size = (256,256,3)

Dataset_Path = '/kaggle/input/gan-getting-started'
kernel_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
gamma_init=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.02)
buffersize=256
batch_size=1

Monet_names = os.listdir(Dataset_Path + '/monet_jpg/')
Monet_paths = tf.io.gfile.glob(str(Dataset_Path + '/monet_jpg/' + '*.jpg'))
#print(tf.io.glob.glob(Dataset_Path + '/monet_frec/*.tfrec'))
#for i in tf.io.gfile(Dataset_Path + '/monet_frec/*.tfrec'):
#    Monet_names.append(Dataset_Path + i)
Photo_names = os.listdir(Dataset_Path + '/photo_jpg/')
Photo_paths = tf.io.gfile.glob(str(Dataset_Path + '/photo_jpg/' + '*.jpg'))
#Monet_names = tf.io.gfile.glob(str(Dataset_Path + '/monet_frec/*.tfrec'))
print(len(Monet_names))
print(Monet_names[0])
print(len(Photo_names))


300
f4413e97bd.jpg
7038


This model is meant to use a generative adversarial networks to build a discriminator between real Monet paintings and photos manipulated to look like Monet paintings, and a generator that is meant to turn photos into Monet-style images. There are 300 Monet paintings and 7038 photographs. All images are 256x256x3, and therefore there is no need to clean the datasets.

In [19]:
#Gotten from https://www.kaggle.com/code/sai10py/gans-art-creation
def decode_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = (tf.cast(img, tf.float32)/127.5)-1
    img = tf.reshape(img, [256, 256, 3])
    return img

Monet_Tensor = tf.data.Dataset.from_tensor_slices(Monet_paths)
Photo_Tensor = tf.data.Dataset.from_tensor_slices(Photo_paths)

Monet_data = Monet_Tensor.map(decode_img, num_parallel_calls=tf.data.AUTOTUNE)
Photo_data = Photo_Tensor.map(decode_img, num_parallel_calls = tf.data.AUTOTUNE)
#Monet_dataset = Monet_Tensor.map(mpimg.imread(Monet_names))
#Photo_dataset = Photo_Tensor.map(mpimg.imread(Photo_names))
#print(tf.shape(Monet_Tensor))

monet_data = Monet_data.shuffle(len(Monet_paths)).batch(1).prefetch(tf.data.AUTOTUNE)
photo_data = Photo_data.shuffle(len(Photo_paths)).batch(1).prefetch(tf.data.AUTOTUNE)

In [20]:
#Monet_Train_images = tf.data.Dataset.from_tensor_slices()

#Monet_train_images = []
#for i in Monet_names_train:
    #temp_image = torchvision.transforms.ToTensor()
#    Monet_train_images.append(tf.constant(mpimg.imread(Dataset_Path + '/monet_jpg/' + str(i)),shape=(1,256,256,3)))

#Monet_test_images = []
#for i in Monet_names_test:
#    Monet_test_images.append(tf.constant(mpimg.imread(Dataset_Path + '/monet_jpg/' + str(i)),shape=(1,256,256,3)))

#Photo_train_images = []
#for i in Photo_names_train:
#    Photo_train_images.append(tf.constant(mpimg.imread(Dataset_Path + '/photo_jpg/' + str(i)),shape=(1,256,256,3)))

#Photo_test_images = []
#for i in Photo_names_test:
#    Photo_test_images.append(tf.constant(mpimg.imread(Dataset_Path + '/photo_jpg/' + str(i)),shape=(1,256,256,3)))

#print(Photo_images)

In [21]:
#plt.imshow(Monet_Tensor)

In [22]:
#plt.imshow(Photo_train_images[0][0])

I struggled immensely with this project and took a lot of guidance from https://www.kaggle.com/code/sai10py/gans-art-creation to get the dataset into position to load, then saw that cycleGAN is good at training images without a pair. I used most of the example code from https://keras.io/examples/generative/cyclegan/. The images have been split into train and test datasets for both paintings and photos.

In [23]:
#Setting Up Dataset to run through GAN, inspired by https://www.kaggle.com/code/victorsullivan/i-m-something-of-a-painter-myself

#kernel_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

#gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

#batch_size=1

#Monet_train_images = (Monet_train_images)
#Photo_train_images = (Photo_train_images)

#Monet_test_images = (Monet_test_images)
#Photo_test_images = (Photo_test_images)

#_, ax = plt.subplots(4, 2, figsize=(10, 15))
#for i, samples in enumerate(zip(Monet_train_images[0][0:4][0:255][0:255], Photo_train_images[0][0:4][0:255][0:255])):
#    ax[i, 0].imshow(samples[0])
#    ax[i,1].imshow(samples[1])
#plt.show()
#gan_train_dataset = tf.data.Dataset.zip((Monet_train_images, Photo_train_images))

#gan_test_dataset = tf.data.Dataset.zip((Monet_test_images, Photo_test_images))
#def make_dataset(monet_files, photo_files, batch_size=1):
    

In [24]:
#_, ax = plt.subplots(4, 2, figsize=(10,15))
#for i, samples in enumerate(zip(Monet_data[0][0:4][0:255][0:255], Photo_data[0][0:4][0:255][0:255])):
#    Monet = samples[0]
#    Photo = samples[1]
#    ax[i,0].imshow(Monet)
#    ax[i,1].imshow(Photo)

#plt.show()

In [25]:
#Discriminator Model interpolated from https://keras.io/examples/generative/dcgan_overriding_train_step/
monet_discriminator = tf.keras.Sequential()
monet_discriminator.add(tf.keras.layers.InputLayer(shape=(256, 256, 3)))
monet_discriminator.add(tf.keras.layers.Conv2D(256, kernel_size=4, strides=2, padding="same", kernel_initializer=kernel_init))
monet_discriminator.add(tf.keras.layers.LeakyReLU(negative_slope=0.2))
monet_discriminator.add(tf.keras.layers.Conv2D(512, kernel_size=4, strides=2, padding="same", kernel_initializer=kernel_init))
monet_discriminator.add(tf.keras.layers.LeakyReLU(negative_slope=0.2))
monet_discriminator.add(tf.keras.layers.Conv2D(512, kernel_size=4, strides=2, padding="same", kernel_initializer=kernel_init))
monet_discriminator.add(tf.keras.layers.LeakyReLU(negative_slope=0.2))
monet_discriminator.add(tf.keras.layers.Flatten())
monet_discriminator.add(tf.keras.layers.Dropout(0.2))
monet_discriminator.add(tf.keras.layers.Dense(1, activation="sigmoid"))

monet_discriminator.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_40 (Conv2D)                   │ (None, 128, 128, 256)       │          12,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 128, 128, 256)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_41 (Conv2D)                   │ (None, 64, 64, 512)         │       2,097,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_15 (LeakyReLU)           │ (None, 64, 64, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_42 (Conv2D)                   │ (None, 32, 32, 512)         │       4,194,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_16 (LeakyReLU)           │ (None, 32, 32, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 524288)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 524288)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │         524,289 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,829,313 (26.05 MB)

 Trainable params: 6,829,313 (26.05 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
#Generator Model interpolated from https://keras.io/examples/generative/dcgan_overriding_train_step/
latent_dim = 512

monet_generator=tf.keras.Sequential()
monet_generator.add(tf.keras.layers.InputLayer(shape=(32, 256, 3)))
#monet_generator.add(keras.layers.Dense(32*32*512))
#monet_generator.add(keras.layers.Reshape((32, 32, 512)))
monet_generator.add(tf.keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", kernel_initializer=kernel_init))
monet_generator.add(tf.keras.layers.LeakyReLU(negative_slope=0.2))
monet_generator.add(tf.keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", kernel_initializer=kernel_init))
monet_generator.add(tf.keras.layers.LeakyReLU(negative_slope=0.2))
monet_generator.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same", kernel_initializer=kernel_init))
monet_generator.add(tf.keras.layers.LeakyReLU(negative_slope=0.2))
monet_generator.add(tf.keras.layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid", kernel_initializer=kernel_init))

monet_generator.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_transpose_7 (Conv2DTranspose) │ (None, 64, 512, 64)         │           3,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_17 (LeakyReLU)           │ (None, 64, 512, 64)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_8 (Conv2DTranspose) │ (None, 128, 1024, 128)      │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_18 (LeakyReLU)           │ (None, 128, 1024, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_9 (Conv2DTranspose) │ (None, 256, 2048, 256)      │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_19 (LeakyReLU)           │ (None, 256, 2048, 256)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_43 (Conv2D)                   │ (None, 256, 2048, 3)        │          19,203 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 678,083 (2.59 MB)

 Trainable params: 678,083 (2.59 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
#Inspired from https://keras.io/examples/generative/cyclegan/
class Reflection_Padding2D(layers.Layer):
    def __init__(self, padding=(1,1), **kwargs):
        self.padding = tuple(padding)
        super().__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [ [0,0], [padding_height, padding_height], [padding_width, padding_width], [0,0]]
        return ops.pad(input_tensor, padding_tensor, mode="REFLECT")

def residual_block(x, activation, kernel_initializer=kernel_init, kernel_size=(3,3), strides=(1,1), padding="valid", gamma_initializer = gamma_init, use_bias=False):
    dim = x.shape[-1]
    input_tensor = x

    x = Reflection_Padding2D()(input_tensor)
    x = layers.Conv2D(dim, kernel_size, strides=strides, kernel_initializer=kernel_initializer, padding=padding, use_bias = use_bias,)(x)
    x = tf.keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)
    x = layers.add([input_tensor, x])
    return x

def downsample(x, filters, activation, kernel_initializer=kernel_init, kernel_size=(3,3), strides=(2,2),padding="same",gamma_initializer=gamma_init, use_bias=False):
    x = layers.Conv2D(filters, kernel_size, strides=strides, kernel_initializer=kernel_initializer, padding=padding, use_bias = use_bias,)(x)
    x = tf.keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x

def upsample(x, filters, activation, kernel_size=(3,3),strides=(2,2), padding="same", kernel_initializer=kernel_init, gamma_initializer=gamma_init, use_bias=False):
    x = layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding, kernel_initializer=kernel_initializer, use_bias=use_bias,)(x)
    x = tf.keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x

In [28]:
#Inspired from https://keras.io/examples/generative/cyclegan/
def get_resnet_generator(filters=64, num_downsampling_blocks=2, num_residual_blocks=9, num_upsample_blocks=2, gamma_initializer=gamma_init, name=None):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = Reflection_Padding2D(padding=(3,3))(img_input)
    x = layers.Conv2D(filters, (7,7), kernel_initializer=kernel_init, use_bias=False)(x)
    x = tf.keras.layers.GroupNormalization(groups=1, gamma_initializer=gamma_initializer)(x)
    x = layers.Activation("relu")(x)

    #Downsampling
    for _ in range(num_downsampling_blocks):
        filters *= 2
        x = downsample(x, filters=filters, activation=layers.Activation("relu"))

    #Residual Blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.Activation("relu"))

    #Upsampling
    for _ in range(num_upsample_blocks):
        filters //=2
        x = upsample(x, filters, activation=layers.Activation("relu"))

    #Final block
    x = Reflection_Padding2D(padding=(3,3))(x)
    x = layers.Conv2D(3, (7,7), padding="valid")(x)
    x = layers.Activation("tanh")(x)

    model = tf.keras.models.Model(img_input, x, name=name)
    return model

In [29]:
def get_discriminator(filters=64, kernel_initializer=kernel_init, num_downsampling=3, name=None):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = layers.Conv2D(filters,(4,4),strides=(2,2),padding="same",kernel_initializer=kernel_initializer,)(img_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters=filters
    for num_downsample_block in range(3):
        num_filters *=2
        if num_downsample_block<2:
            x = downsample(x, filters=num_filters, activation=layers.LeakyReLU(0.2),kernel_size=(4,4),strides=(2,2),)
        else:
            x = downsample(x, filters=num_filters, activation=layers.LeakyReLU(0.2),kernel_size=(4,4),strides=(1,1),)

    x = layers.Conv2D(1, (4,4), strides=(1,1), padding="same", kernel_initializer=kernel_initializer)(x)
    model = tf.keras.models.Model(inputs=img_input, outputs=x, name=name)
    return model

In [30]:
#Get generators and discriminators
gen_F = get_resnet_generator(name="generator_F")
gen_G = get_resnet_generator(name="generator_G")

disc_X = get_discriminator(name="discriminator_X")
disc_Y = get_discriminator(name="discriminator_Y")

In [39]:
#GAN model to override train_step interpolated from https://keras.io/examples/generative/dcgan_overriding_train_step/
#%%Need to work on removing random and replacing with photo%%
class monet_GAN(tf.keras.Model):
    def __init__(self, generator_G, generator_F, discriminator_X, discriminator_Y, lambda_cycle=10.0, lambda_identity=0.5):
        super().__init__()
        self.gen_G = generator_G
        self.gen_F = generator_F
        self.disc_X = discriminator_X
        self.disc_Y = discriminator_Y
        self.lambda_cycle = lambda_cycle
        self.lambda_identity = lambda_identity
        #self.seed_generator = keras.random.SeedGenerator(1996)

    def call(self, inputs):
        return(self.disc_X(inputs), self.disc_Y(inputs),self.gen_G(inputs), self.gen_F(inputs))
    
    def compile(self, gen_G_opt, gen_F_opt, disc_X_opt, disc_Y_opt, gen_loss_func, disc_loss_func):
        super().compile()
        self.gen_G_opt = gen_G_opt
        self.gen_F_opt = gen_F_opt
        self.disc_X_opt = disc_X_opt
        self.disc_Y_opt = disc_Y_opt
        self.gen_loss_func = gen_loss_func
        self.disc_loss_func = disc_loss_func
        self.cycle_loss_func = tf.keras.losses.MeanAbsoluteError()
        self.identity_loss_func = tf.keras.losses.MeanAbsoluteError()

    #@property
    #def metrics(self):
    #    return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, batch_data):
        # Sample random points in the latent space
        #batch_size = ops.shape(images[0])[0]
        #random_latent_vectors = keras.random.normal(
        #    shape=(batch_size, self.latent_dim), seed=self.seed_generator
        #)
        #print(batch_data.size)
        #monet_img = batch_data[:][0:225][0:255][0:255][0:2]
        #photo_img = batch_data[:][0:5278][0:255][0:255][0:2]
        #batch_size = tf.shape(real_monet)[0]
        # Decode them to photo images
        real_x, real_y = batch_data
        #generated_images = self.generator(photo_img)

        # Combine them with real images
        #combined_images = ops.concatenate([monet_img, photo_img], axis=0)

        # Assemble labels discriminating real from fake images
        #labels = ops.concatenate(
        #    [ops.ones((batch_size, 1)), ops.zeros((batch_size, 1))], axis=0
        #)
        # Add random noise to the labels - important trick!
        #labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape(persistent=True) as tape:
            fake_y = self.gen_G(real_x, training=True)
            fake_x = self.gen_F(real_y, training=True)
            cycled_x = self.gen_F(fake_y, training=True)
            cycled_y = self.gen_G(fake_x, training=True)

            #Identity mapping
            same_x = self.gen_F(real_x, training=True)
            same_y = self.gen_G(real_y, training=True)

            disc_real_x = self.disc_X(real_x, training=True)
            disc_fake_x = self.disc_X(fake_x, training=True)

            disc_real_y = self.disc_Y(real_y, training=True)
            disc_fake_y = self.disc_Y(fake_y, training=True)

            #Gen adv loss
            gen_G_loss = self.gen_loss_func(disc_fake_y)
            gen_F_loss = self.gen_loss_func(disc_fake_x)

            #Gen cycle loss
            cycle_loss_G = self.cycle_loss_func(real_y, cycled_y)*self.lambda_cycle
            cycle_loss_F = self.cycle_loss_func(real_x, cycled_x)*self.lambda_cycle

            #Gen Identity loss
            id_loss_G = (self.identity_loss_func(real_y, same_y)*self.lambda_cycle*self.lambda_identity)
            id_loss_F = (self.identity_loss_func(real_x, same_x)*self.lambda_cycle*self.lambda_identity)

            #Total Gen loss
            total_loss_G = gen_G_loss+cycle_loss_G+id_loss_G
            total_loss_F = gen_F_loss+cycle_loss_F+id_loss_F

            #Total Discriminator loss
            disc_X_loss = self.disc_loss_func(disc_real_x, disc_fake_x)
            disc_Y_loss = self.disc_loss_func(disc_real_y, disc_fake_y)

        #Get the gradients for generators
        grads_G = tape.gradient(total_loss_G, self.gen_G.trainable_variables)
        grads_F = tape.gradient(total_loss_F, self.gen_F.trainable_variables)

        #Get gradients for discriminators
        disc_X_grads = tape.gradient(disc_X_loss, self.disc_X.trainable_variables)
        disc_Y_grads = tape.gradient(disc_Y_loss, self.disc_Y.trainable_variables)

        #++JG
        #print(grads_G)
        #print(self.grads_G.trainable_variables)
        #Tried removing self from grads_G
        #--JG
        #Update the weights of the generators
        self.gen_G_opt.apply_gradients(zip(grads_G, self.gen_G.trainable_variables))
        self.gen_F_opt.apply_gradients(zip(grads_F, self.gen_F.trainable_variables))

        #Update weights of discriminators
        self.disc_X_opt.apply_gradients(zip(disc_X_grads, self.disc_X.trainable_variables))
        self.disc_Y_opt.apply_gradients(zip(disc_Y_grads, self.disc_Y.trainable_variables))

        return {
            "gen_1_loss": total_loss_G,
            "gen_2_loss": total_loss_F,
            "disc_1_loss": disc_X_loss,
            "disc_2_loss": disc_Y_loss,
        }

In [40]:
#Training the model
adv_loss_func = tf.keras.losses.MeanSquaredError()

def gen_loss_func(fake):
    fake_loss = adv_loss_func(ops.ones_like(fake),fake)
    return fake_loss

def disc_loss_func(real, fake):
    real_loss = adv_loss_func(ops.ones_like(real),real)
    fake_loss = adv_loss_func(ops.zeros_like(fake),fake)
    return (real_loss + fake_loss) * 0.5

Monet_gan_model = monet_GAN(generator_F = gen_F, generator_G = gen_G, discriminator_X = disc_X, discriminator_Y = disc_Y)

Monet_gan_model.compile(
    gen_F_opt=tf.keras.optimizers.SGD(learning_rate=2e-4),
    gen_G_opt=tf.keras.optimizers.SGD(learning_rate=2e-4),
    disc_X_opt=tf.keras.optimizers.SGD(learning_rate=2e-4),
    disc_Y_opt=tf.keras.optimizers.SGD(learning_rate=2e-4),
    gen_loss_func = gen_loss_func,
    disc_loss_func = disc_loss_func,
)

#Callbacks
epochs = 1
Monet_gan_model.fit(
    tf.data.Dataset.zip((monet_data, photo_data)),
    epochs=1,
)

300/300 ━━━━━━━━━━━━━━━━━━━━ 5615s 19s/step - disc_1_loss: 0.4294 - disc_2_loss: 0.4291 - gen_1_loss: 8.0279 - gen_2_loss: 6.7344


Due to my struggles I ran out of time with this project and need to submit this model. I would approach changing the number of epochs and changing the optimizers and their learning rate.

In [ ]:
! mkdir ../images
    
i = 1
for img in photo_data:
    prediction = Monet_gan_model.gen_G(img, training=False)[0].numpy()
    prediction = (prediction*127.5 + 127.5).astype(np.uint8)
    im = PIL.Image.fromarray(prediction)
    im.save("../images" + str(i) + ".jpg")

shutil.make_archive("/kaggle/working/images","zip","/kaggle/images")

mkdir: cannot create directory ‘../images’: File exists


My submission got a score of %%. I learned that coding all night is a good way to make careless mistakes and that the TA for this course is extremely helpful